### comment preprocessing
- 숫자는 욕설로 만들 수 있음
- 초성만으로 이루어진 단어또한 비웃음이나 욕설로 만들 수 있음
- 이모티콘은 실제 사용에서는 어떨지 모르나 데이터로 들어올 때는 학습에 방해될 것으로 예상됨(UNK 토큰으로 판단될 수 있다고 생각함)

In [8]:
""" 
    Reference : https://gist.github.com/kse0202/9d3d8d519170064cefdd12fcb718afa0
"""
import re
import numpy as np
from tqdm import tqdm, trange

def preprocessing(dataset):
    
    for i in trange(len(dataset)):
        comment = dataset.iloc[i, 0]
        # checking nan
        if comment != comment:
            continue
        
        tmp = comment
        comment = re.sub(r"[“”‘’\"\']", r"\'", comment)
        comment = re.sub(r"[〈<＜「≪《『]", "<", comment)
        comment = re.sub(r"[〉>＞」≫》』]", ">", comment)
        comment = re.sub(r'[‥…]+', r'...', comment)
        comment = re.sub(r'[\?¿？]+', r'\?', comment)
        comment = re.sub(r'[!¡！]+', r'!', comment)
        comment = re.sub(r"([^\-—_=+,\./<>?\[\]{};:\'\"!@#$%\^&*\(\)₩`´~\|\\ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9ぁ-ゔゞァ-・ヽヾ゛゜ー一-龯\u3000-\u303F\u3400-\u4DBF\u4E00-\u9FFF\s]+)", r'', comment)
        comment = re.sub(r"\s+", r" ", comment)
        if len(comment) <= 1 and tmp != comment:
            comment = np.nan
        elif len(comment) > 500:
            comment = np.nan
        
        dataset.iloc[i, 0] = comment
        
    print("checking nan")
    print(sum(dataset['text'].isna()), "number of nan exist")
    dataset = dataset[dataset['text'].notna()]
    print("checking null")
    print(sum(dataset['text'].isnull()), "number of null exist")
    dataset = dataset[dataset['text'].notnull()]
    
    return dataset

In [10]:
import re
import numpy as np
from tqdm import tqdm, trange

quotes = re.compile(r"[“”‘’\"\']")
l_bracket = re.compile(r"[〈<＜「≪《『]")
r_bracket = re.compile(r"[〉>＞」≫》』]")
dots = re.compile(r'[‥…]+')
question = re.compile(r'[\?¿？]+')
exclamation = re.compile(r'[!¡！]+')
remainders = re.compile(r"([^\-—_=+,\./<>?\[\]{};:\'\"!@#$%\^&*\(\)₩`´~\|\\ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9ぁ-ゔゞァ-・ヽヾ゛゜ー一-龯\u3000-\u303F\u3400-\u4DBF\u4E00-\u9FFF\s]+)")
multiple_spaces = re.compile(r"\s+")

def preprocessing(dataset):
    
    for i in trange(len(dataset)):
        comment = dataset.iloc[i, 0]
        # checking nan
        if comment != comment:
            continue
        
        tmp = comment
        comment = quotes.sub(r"\'", comment)
        comment = l_bracket.sub("<", comment)
        comment = r_bracket.sub(">", comment)
        comment = dots.sub(r'...', comment)
        comment = question.sub(r'\?', comment)
        comment = exclamation.sub(r'!', comment)
        comment = remainders.sub(r'', comment)
        comment = multiple_spaces.sub(r" ", comment)
        if len(comment) <= 1 and tmp != comment:
            comment = np.nan
        elif len(comment) > 500:
            comment = np.nan
        
        dataset.iloc[i, 0] = comment
        
    print("checking nan")
    print(sum(dataset['text'].isna()), "number of nan exist")
    dataset = dataset[dataset['text'].notna()]
    print("checking null")
    print(sum(dataset['text'].isnull()), "number of null exist")
    dataset = dataset[dataset['text'].notnull()]
    
    return dataset

In [6]:
import pandas as pd

curse_dataset = pd.read_csv('curse.tsv', sep='\t')
beep_dataset = pd.read_csv('beepData.tsv', sep='\t')
twitch_dataset = pd.read_csv('chatData.tsv', sep='\t')
train_dataset = pd.concat([curse_dataset[['text']], beep_dataset, twitch_dataset[['text']]], ignore_index=True)
train_dataset = train_dataset[train_dataset['text'].notna()]
train_dataset = train_dataset[train_dataset['text'].notnull()]
print(len(train_dataset))
train_dataset.head()

1457392


,text
0,좌배 까는건 ㅇㅂ
1,집에 롱 패딩만 세 개다 10년 더 입어야지 ㅋㅋ
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아
3,세탁이라고 봐도 된다
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ


In [11]:
train_df = preprocessing(train_dataset)
len(train_df)

100%|██████████| 1457392/1457392 [03:53<00:00, 6251.62it/s]


checking nan
3 number of nan exist
checking null
0 number of null exist


1457389

In [7]:
import random

def punctuation(dataset):
    
    """punctuation preprocessing."""
    """텍스트 길이의 10%~20%를 punctuation 삽입하여 모델이 robust하도록 한다."""
    new_dataset = []
    punc = ['.',',',"'",';','/','-','~','!','@','?','^',' ']
    
    for i in trange(len(dataset)):
        text = dataset[i]
        if len(text) <= 30 and random.random() < 0.1: # 0.3 확률로 띄어쓰기 없애기 (길이 30 이하 텍스트만)
            text = ''.join(text.split())
            
        if random.random() < 0.3: # 0.4 확률로 punctuation 추가
            punc_size = random.randint(max(len(text)//20, 3), max(len(text)//10, 3)) # 모든 텍스트에 최소 3개는 들어가도록
            text = list(text)
            for _ in range(punc_size):
                txt_rnd = random.randint(0, len(text)-1)
                punc_rnd = random.randint(0, len(punc)-1)
                text.insert(txt_rnd, punc[punc_rnd])
            new_dataset.append(''.join(text).replace("  ", " "))
        else:
            new_dataset.append(text)
    return new_dataset

In [8]:
text = train_df['text'].to_list()
text_list = []
for _ in range(100):
    text_list += punctuation(text)
len(text_list)

100%|██████████| 1457389/1457389 [00:05<00:00, 257118.28it/s]


145738900

In [9]:
text_list[:10]

['좌-배 ~까는건 ㅇㅂ',
 "집에,' 롱 패딩만 세 개다 10년 더' 입어야지 ㅋㅋ",
 '개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아',
 '세탁이라고 봐도 된다',
 '애새끼가 초딩도 아니고 ㅋㅋㅋㅋ',
 '731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 벌리는 한국에 기를 써서 진출하려고 하지조센남자들은 또 이쁜여자만 보면 사족을 못쓰며 공주대접해주는 놈들이니',
 '재앙이한건햇노',
 '글쓴이 와꾸 승리에 비하면 방사능 피폭 원숭이 일듯',
 '마 씨발련 아 몇평이고 맷개드갔노 니 대하이햄하고 해밨나',
 '은행에 대출 상담 받으러 가보면 직업의 귀천 바로 알려줌']

In [10]:
label_to_num = dict(
    none = 0, label = 1
)
label_to_num

{'none': 0, 'label': 1}

In [11]:
num_to_label = {}
for k in label_to_num.keys():
    num_to_label[label_to_num[k]]=k
num_to_label

{0: 'none', 1: 'label'}

In [6]:
# import pickle
# with open('./label_to_num', 'rb') as f:
#     label_to_num = pickle.load(f)

# with open('./num_to_label', 'rb') as f:
#     num_to_label = pickle.load(f)

In [7]:
# def tocsv(dataset):
#     df = pd.DataFrame(columns=['text','label'])
#     for i in trange(len(dataset)):
#         df=df.append(
#             dict(
#                 text=dataset['text'][i],
#                 label=(0 if dataset['curse'][i]==0 else 1)
#             )
#             ,ignore_index=True)
#     return df

In [8]:
# train_df = tocsv(train_dataset)
# eval_df = tocsv(eval_dataset)

In [10]:
# print('train dataset none 개수 : {}, curse 개수 : {}'.format(list(train_df['label']).count(0), list(train_df['label']).count(1)))
# print('eval dataset none 개수 : {}, hate 개수 : {}'.format(list(eval_df['label']).count(0), list(eval_df['label']).count(1)))

In [12]:
train_df = train_df.drop(index=train_df[train_df['text'] == ''].index)

In [12]:
train_df.to_csv('./train.tsv', sep='\t')
# eval_df.to_csv('./eval_data.csv')

토크나이저를 학습하지 말고 사전학습된 토크나이저를 사용하자.  
  
왜냐하면 문장내 어떤 단어가 올지 모르겠음 -> 단어가 더 많은 사전학습된 토크나이저가 유리하다고 판단  
  
또한, 전처리 부분에 있어서도 사전학습된 토크나이저가 성능이 좋은듯  
  
하지만 욕설에 대해서 토크나이징이 어떨지는 테스트해야할듯

In [15]:
# adding = []
# unk_ = []
# for text in tqdm(list(train_df['text'])):
#     tokens = tokenizer(text)['input_ids']
#     if 1 in tokens:
#         tokenized = tokenizer.decode(tokens)[6:-6]
#         # print(text, tokenized)
#         tokenized = re.sub("\[UNK\]", r"(.+)", tokenized)
#         try:
#             unknown = re.match(tokenized, text)
#         except:
#             unknown = None
#         if unknown:
#             adding.append(unknown.group(1).strip())
#         unk_.append([text, tokenizer.decode(tokens)])
# unk_

In [14]:
# txt_file = '\n'.join(train_df.loc[:,"text"].values.tolist())
# len(txt_file)

19333506

In [ ]:
txt_file = '\n'.join(text_list)
len(txt_file)

In [ ]:
with open("./sentence.txt", "w", encoding="UTF-8") as f:
    f.write(txt_file)

In [2]:
with open("./sentence.txt", "r", encoding="UTF-8") as f:
    asdfg = f.read().splitlines()
    
len(asdfg)

145738900

In [3]:
from tokenizers import BertWordPieceTokenizer

special = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
unused = [f'[unused{i}]' for i in range(300)]

# load sentences
tokenizer = BertWordPieceTokenizer(
    vocab=None,
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=False,
    lowercase=False,
    wordpieces_prefix="##",
)

limit_alphabet = 10000
vocab_size = 30000

tokenizer.train(
    files='./sentence.txt', # 문장들을 모아놓음
    vocab_size=vocab_size,
    min_frequency=2,
    special_tokens = special + unused,
    show_progress=True,
    limit_alphabet=limit_alphabet,
)

tokenizer.save("./tokenizer.json", pretty=True)  # save tokenizer.json, pretty=True로 두시면 json 형식이 보기좋게 저장됩니다
tokenizer.save_model('./')  # save vocab.txt

# from pretrained("./vocab.txt")

['./vocab.txt']

In [4]:
tokenizer?

Type:        BertWordPieceTokenizer
String form: Tokenizer(vocabulary_size=30000, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token <...> text=True, handle_chinese_chars=True, strip_accents=False, lowercase=False, wordpieces_prefix=##)
File:        /opt/conda/envs/lightweight/lib/python3.7/site-packages/tokenizers/implementations/bert_wordpiece.py
Docstring:   Bert WordPiece Tokenizer 


In [11]:
with open("./vocab.txt", "r") as f:
    lines = f.read().splitlines()
len(lines)

30000

In [ ]:
# adding_prep = []

# for token in tqdm(adding):
    
#     token = re.sub(r"[ㄱ-ㅎㅏ-ㅣ0-9]+", r"", token)
#     token = re.sub(r"[ㄱ-ㅎㅏ-ㅣ]+", r"", token)
#     token = re.sub(r"[ㄱ-ㅎㅏ-ㅣ]+", r"", token)
    
#     if re.fullmatch(r"[ぁ-ゔゞァ-・ヽヾ゛゜ー一-龯]+", token): # w, 숫자, 일본어만 있는 문장 제거
#         continue
        
#     elif re.fullmatch(r"[\u3000-\u303F\u3400-\u4DBF\u4E00-\u9FFF\d\s]+", token): # 영어, 숫자, 중국어만 있는 문장 제거
#         continue
    
#     rep = check_repeat(token)
#     if rep: token = rep
    
#     while re.search(r"(.)\1{4,}", fr"{user_chat}"):  # 4번 이상 반복되는 글자 3번만 반복되도록 수정
#         repeated = re.search(r"(.)\1{4,}", fr"{user_chat}").group()
#         if repeated[0] == '^':
#             user_chat = re.sub(r'\^+', r'^^^', user_chat)
#         try:
#             user_chat = re.sub(repeated, repeated[0]*3, user_chat)
#         except: break
            
#     user_chat = re.sub(r"\\", "", user_chat) # 백슬래쉬 제거
    
#     chat_log.append((user_id, user_chat))

# len(chat_log)

In [124]:
# adding += ['갠탆앗음', '얼마정도ᄇᆞ는데', 'ㄱㅐ씹죶간이고', '꺼늏다보니', '쓉쒱키는', '黄']

1365

In [ ]:
# tokenizer.add_tokens(adding)
# tokenizer.get_added_vocab()

## 추가해야 할 단어들
- 갠탆앗음
- 얼마정도ᄇᆞ는데
